<a href="https://colab.research.google.com/github/u5638928/u5638928-DataScience-GenAI-Submissions/blob/main/Copy_of_2_02_Joining_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](https://drive.google.com/uc?export=view&id=1xqQczl0FG-qtNA2_WQYuWePW9oU8irqJ)

# 2.02 Pandas: Joining and merging dataframes
### Why Combine Dataframes?
Quite often in practical applications we have data coming from multiple sources such as different files, database tables or IoT devices. This is particularly likely when working in data science and artificial intelligence. However, typically we want to create a single dataframe to run our analysis and/or models. To achieve this commonly we would want to combine dataframes together.

Joining data is a large topic and an important one in database administration and data engineering. We will look at this at a relatively high level, althout the interested reader can certainly explore this in much more depth.

Let us begin by using the orders_df from the previous session.

In [ ]:
import pandas as pd
import numpy as np

orders = {'o10001':{'date':'2024/01/10', 'product':'Hoodie', 'quantity':'1'},
            'o10002':{'date':'2024/01/13', 'product':'Tote bag', 'quantity':'2'},
            'o10003':{'date':'2024/01/14', 'product':'Pencil', 'quantity':'10'},
            'o10004':{'date':'2024/01/15', 'product':'T-shirt', 'quantity':'2'}
}

orders_df = pd.DataFrame(orders)
orders_df

,o10001,o10002,o10003,o10004
date,2024/01/10,2024/01/13,2024/01/14,2024/01/15
product,Hoodie,Tote bag,Pencil,T-shirt
quantity,1,2,10,2


In this scenario, we also have some data about these products from our operations team. The data is currently in a dictionary and can be transformed in the usual fashion:

In [ ]:
products = {'123':{'name':'Hoodie', 'cost_price':12.12, 'sale_price':15.00},
            '124':{'name':'Tote bag', 'cost_price':2.15, 'sale_price':9.99},
            '125':{'name':'Pencil', 'cost_price':22.45, 'sale_price':49.99},
            '126':{'name':'T-shirt', 'cost_price':0.45, 'sale_price':2.99},
            '127':{'name':'Honours degree', 'cost_price':0.78, 'sale_price':1.49}
}

products_df = pd.DataFrame(products)
products_df

,123,124,125,126,127
name,Hoodie,Tote bag,Pencil,T-shirt,Honours degree
cost_price,12.12,2.15,22.45,0.45,0.78
sale_price,15.0,9.99,49.99,2.99,1.49


### Joining Dataframes using Merge
What we want to do is join these datasets together. However, currently the Dataframes are both the wrong way round. We would want our rows in orders_df to be the order ID, and the rows in products_df to be the product ID. We can transform the data using the Dataframe.transpose function:

In [ ]:
orders_df = orders_df.transpose()
products_df = products_df.transpose()

Transpose basically flips the axes of the Dataframe so the current columns become rows and vice versa. Note, _Dataframe.transpose_ creates a new object by default. We use the "orders_df = ..." element at the start to reassign the variable and make orders_df equal the new, transposed version.

In [ ]:
orders_df

,date,product,quantity
o10001,2024/01/10,Hoodie,1
o10002,2024/01/13,Tote bag,2
o10003,2024/01/14,Pencil,10
o10004,2024/01/15,T-shirt,2


With this in place we can join the two Dataframes. There are several decisions that need to be taken when joining two datasets. Firstly, there are multiple ways to join data in pandas but the most appropriate here would seem to be using the _merge( )_ function. These choices will be discussed in more detail later in the notebook. Secondly we need to decide on the element that we will make the join on. In this case, the data that the two dataframes have in common are the product name, labelled “product” in orders_df and as “name” in products_df. We also need to decide the type of join – what do we want to do with rows that are in one set but not in the other? This too is discussed further below. In this case the scenario is to get more details on our orders, so we are joining products_df to orders_df so we will base our rows on the orders_df dataframe. With these choices in mind we get the following code:

In [ ]:
joined_df = orders_df.merge(products_df, how='left', left_on='product', right_on='name')

We’ll unpack this a little more because it is a more complex function. We are creating a new dataframe called joined_df. This will be based on the orders_df being merged with products_df as a ‘left join’. This means we will use the rows of the orders_df not products_df (as orders_df is on the left hand of the command). Finally we need to tell Python which columns we will be joining based on. In this case the left hand dataframe (orders_df) lists the relevant data as “product”, and the right hand (products_df) lists it as “name”.

In [ ]:
joined_df

,date,product,quantity,name,cost_price,sale_price
0,2024/01/10,Hoodie,1,Hoodie,12.12,15.0
1,2024/01/13,Tote bag,2,Tote bag,2.15,9.99
2,2024/01/14,Pencil,10,Pencil,22.45,49.99
3,2024/01/15,T-shirt,2,T-shirt,0.45,2.99


There are several types of joins that can be used here. The main types are shown below. The yellow bit in each Venn diagram represents the bit we would keep if we perform this type of join.

So far we have only looked at a "left join" (labelled in pandas as "left"). It would also be possible to specify an "inner join" ("inner"). This means we only keep records (rows) that are in both datasets. Only the records in both would be included if we use an "inner join". Finally we could use a "full join" ("outer") which means that we keep all the records irrespective of whether they are missing in one of the sides.

![](https://drive.google.com/uc?export=view&id=1e8rPwaCqpnrB9-yIcqKOm2sb6VUPGz_g)

Let’s return to our first merged dataframe (joined_df) as this better fits the description of this problem (we care more about orders in this problem than products). One immediate issue is that we have duplicated columns product and name. Let’s tidy this up by dropping the name column:

In [ ]:
joined_df = joined_df.drop(['name'], axis=1)
joined_df

,date,product,quantity,cost_price,sale_price
0,2024/01/10,Hoodie,1,12.12,15.0
1,2024/01/13,Tote bag,2,2.15,9.99
2,2024/01/14,Pencil,10,22.45,49.99
3,2024/01/15,T-shirt,2,0.45,2.99


The _drop( )_ function can be used to remove columns or rows. axis=1 indicates that we want to a drop a column(s), axis=0 would mean we were dropping a row(s). Note as well that although we are only dropping one column we still need to present the column name inside a list as this is what the function expects (a list).

### EXERCISE
In the above example we merged using a left join. How would different types of joins effect the final dataframe? Experiment with inner join, right join and outer join. What impact does it have on the data retained? Are the results what you expected them to be?

## Experiment with Inner Join

### Subtask:
Perform an inner merge of 'orders_df' and 'products_df' on 'product' and 'name' columns, and display the result. Then, drop the 'name' column for consistency.


In [ ]:
inner_joined_df = orders_df.merge(products_df, how='inner', left_on='product', right_on='name')
print("DataFrame after inner merge:")
print(inner_joined_df)

inner_joined_df = inner_joined_df.drop(['name'], axis=1)
print("\nDataFrame after dropping 'name' column:")
print(inner_joined_df)

DataFrame after inner merge:
         date   product quantity      name cost_price sale_price
0  2024/01/10    Hoodie        1    Hoodie      12.12       15.0
1  2024/01/13  Tote bag        2  Tote bag       2.15       9.99
2  2024/01/14    Pencil       10    Pencil      22.45      49.99
3  2024/01/15   T-shirt        2   T-shirt       0.45       2.99

DataFrame after dropping 'name' column:
         date   product quantity cost_price sale_price
0  2024/01/10    Hoodie        1      12.12       15.0
1  2024/01/13  Tote bag        2       2.15       9.99
2  2024/01/14    Pencil       10      22.45      49.99
3  2024/01/15   T-shirt        2       0.45       2.99


**Reasoning**:
The next step is to perform a right merge of 'orders_df' and 'products_df', display the result, then drop the redundant 'name' column for consistency, and display the DataFrame again, as outlined in the main task instructions.



In [ ]:
right_joined_df = orders_df.merge(products_df, how='right', left_on='product', right_on='name')
print("DataFrame after right merge:")
print(right_joined_df)

right_joined_df = right_joined_df.drop(['name'], axis=1)
print("\nDataFrame after dropping 'name' column:")
print(right_joined_df)

DataFrame after right merge:
         date   product quantity            name cost_price sale_price
0  2024/01/10    Hoodie        1          Hoodie      12.12       15.0
1  2024/01/13  Tote bag        2        Tote bag       2.15       9.99
2  2024/01/14    Pencil       10          Pencil      22.45      49.99
3  2024/01/15   T-shirt        2         T-shirt       0.45       2.99
4         NaN       NaN      NaN  Honours degree       0.78       1.49

DataFrame after dropping 'name' column:
         date   product quantity cost_price sale_price
0  2024/01/10    Hoodie        1      12.12       15.0
1  2024/01/13  Tote bag        2       2.15       9.99
2  2024/01/14    Pencil       10      22.45      49.99
3  2024/01/15   T-shirt        2       0.45       2.99
4         NaN       NaN      NaN       0.78       1.49


**Reasoning**:
The next step is to perform an outer merge of 'orders_df' and 'products_df', display the result, then drop the redundant 'name' column for consistency, and display the DataFrame again, as outlined in the main task instructions.



In [ ]:
outer_joined_df = orders_df.merge(products_df, how='outer', left_on='product', right_on='name')
print("DataFrame after outer merge:")
print(outer_joined_df)

outer_joined_df = outer_joined_df.drop(['name'], axis=1)
print("\nDataFrame after dropping 'name' column:")
print(outer_joined_df)

DataFrame after outer merge:
         date   product quantity            name cost_price sale_price
0         NaN       NaN      NaN  Honours degree       0.78       1.49
1  2024/01/10    Hoodie        1          Hoodie      12.12       15.0
2  2024/01/14    Pencil       10          Pencil      22.45      49.99
3  2024/01/15   T-shirt        2         T-shirt       0.45       2.99
4  2024/01/13  Tote bag        2        Tote bag       2.15       9.99

DataFrame after dropping 'name' column:
         date   product quantity cost_price sale_price
0         NaN       NaN      NaN       0.78       1.49
1  2024/01/10    Hoodie        1      12.12       15.0
2  2024/01/14    Pencil       10      22.45      49.99
3  2024/01/15   T-shirt        2       0.45       2.99
4  2024/01/13  Tote bag        2       2.15       9.99


### Analysis of Different Join Types

Let's summarize the differences observed with each join type:

1.  **Left Join (Original `joined_df`)**:
    *   **Result**: Contains 4 rows.
    *   **Data Retained**: All rows from `orders_df` are kept. Columns from `products_df` are added where there's a match on 'product' and 'name'. If a product from `orders_df` doesn't have a match in `products_df`, the `products_df` columns would show `NaN` (Not a Number). In our specific case, all products in `orders_df` had a match in `products_df`, so no `NaN` values appeared for `products_df` columns.
    *   **Implication**: Useful when you want to retain all records from your primary (left) DataFrame and augment them with information from another DataFrame, even if some matches don't exist.

2.  **Inner Join (`inner_joined_df`)**:
    *   **Result**: Contains 4 rows.
    *   **Data Retained**: Only retains rows where there is a match in *both* `orders_df` and `products_df` based on the specified columns ('product' and 'name'). Since all products in `orders_df` had a corresponding entry in `products_df` in this specific dataset, the result is identical to the left join.
    *   **Implication**: Best suited when you only care about the intersection of data between two DataFrames. Records that do not have a match in both are excluded.

3.  **Right Join (`right_joined_df`)**:
    *   **Result**: Contains 5 rows.
    *   **Data Retained**: All rows from `products_df` are kept. Columns from `orders_df` are added where there's a match. If a product from `products_df` (e.g., 'Honours degree') doesn't have a match in `orders_df`, the `orders_df` columns (`date`, `product`, `quantity`) show `NaN`.
    *   **Implication**: Ideal when you want to preserve all records from the secondary (right) DataFrame and add matching information from the primary DataFrame. This helps identify entries in the right DataFrame that do not have corresponding entries in the left DataFrame.

4.  **Outer Join (`outer_joined_df`)**:
    *   **Result**: Contains 5 rows.
    *   **Data Retained**: All rows from *both* `orders_df` and `products_df` are kept. `NaN` values appear in columns where there is no match in the corresponding DataFrame. For example, 'Honours degree' from `products_df` has `NaN` for `orders_df` columns, and if there were an order for a product not in `products_df`, its `products_df` columns would be `NaN`.
    *   **Implication**: Used when you need to see all possible combinations and identify all discrepancies or missing data points across both DataFrames. It's the most comprehensive join type.

## Final Task

### Subtask:
Provide a summary of the observations from experimenting with different join types and their impact on the merged DataFrame.


## Summary:

### Q&A
The task implicitly asked to analyze the differences in data retained by inner, right, and outer joins compared to a left join, and discuss their implications.

*   **Differences compared to Left Join:**
    *   **Inner Join**: In this specific dataset, the inner join yielded the same number of rows (4) as the left join because all entries in the left DataFrame (`orders_df`) had a corresponding match in the right DataFrame (`products_df`). It exclusively retains records that have matches in both DataFrames.
    *   **Right Join**: The right join resulted in 5 rows, which is more than the left join (4 rows). It included all records from the right DataFrame (`products_df`), adding rows for entries that had no match in `orders_df` (e.g., 'Honours degree'), where `orders_df` columns showed `NaN`.
    *   **Outer Join**: The outer join also produced 5 rows, similar to the right join in this scenario. It is the most comprehensive, retaining all records from both DataFrames, and showing `NaN` values in columns where a match was not found from the respective side.

*   **Implications:**
    *   **Left Join**: Ideal for retaining all records from the primary (left) DataFrame and augmenting them with information from another, showing `NaN` where no match exists in the right DataFrame.
    *   **Inner Join**: Best suited for finding the common intersection of data, excluding any records that don't have a match in both DataFrames.
    *   **Right Join**: Useful when the integrity of all records in the secondary (right) DataFrame is crucial, and you want to identify which of these records lack corresponding entries in the primary DataFrame.
    *   **Outer Join**: Provides a complete picture by including all records from both DataFrames, highlighting all discrepancies and missing data points across the joined datasets.

### Data Analysis Key Findings
*   The **inner join** of `orders_df` and `products_df` resulted in 4 rows, retaining only records where there was a match in both DataFrames. In this specific dataset, this count was identical to a left join because all products in `orders_df` had corresponding entries in `products_df`.
*   The **right join** produced 5 rows, encompassing all entries from `products_df`. This included an additional row for 'Honours degree' which was present in `products_df` but not in `orders_df`, resulting in `NaN` values for the `date`, `product`, and `quantity` columns from `orders_df`.
*   The **outer join** also yielded 5 rows, combining all records from both `orders_df` and `products_df`. It similarly showed `NaN` values for `orders_df` columns corresponding to 'Honours degree', as it was an unmatched entry from `products_df`.
*   In all merge operations, the redundant 'name' column was successfully dropped, maintaining consistency while preserving the analytical insights of each join type.

### Insights or Next Steps
*   The choice of join type significantly impacts the number of rows and the completeness of data retained, emphasizing the importance of selecting the appropriate join based on the analytical objective and desired data scope (intersection, left-centric, right-centric, or comprehensive).
*   Understanding the implications of `NaN` values introduced by joins like right and outer is crucial for data cleaning and downstream analysis, as they indicate non-matching records and potential data gaps.


Implications to industry. Finance/Consulting. Merging data set.